In [ ]:
import numpy as np
import pandas as pd
import gzip
import json
import re
from pprint import pprint
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
from datetime import datetime
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt


#TASK-1

#**FEATURE EXTRACTION**

In my code I used the given example codes and made little changes on them. I added more metadata columns.

In [ ]:
nltk.download('stopwords')
turkish_stopwords = stopwords.words('turkish')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Load the CSV file
train_classification_df = pd.read_csv('my_training.csv')

# Inspect the data
print(train_classification_df.head())
print(train_classification_df.info())


                   Unnamed: 0                 label
0             cankiriotelavsa                Travel
1                   kademe.tr                  Tech
2              chefsidarbudak                  Food
3               birddetective                   Art
4  diyetisyenseymakavuncuoglu  Health and Lifestyle
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2842 entries, 0 to 2841
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  2842 non-null   object
 1   label       2842 non-null   object
dtypes: object(2)
memory usage: 44.5+ KB
None


In [ ]:
train_classification_df = pd.read_csv("my_training.csv",)
train_classification_df = train_classification_df.rename(columns={'Unnamed: 0': 'user_id', 'label': 'category'})

# Unifying labels
train_classification_df["category"] = train_classification_df["category"].apply(str.lower)
username2_category = train_classification_df.set_index("user_id").to_dict()["category"]

In [ ]:
# stats about the labels
train_classification_df.groupby("category").count()

,user_id
category,
art,198
entertainment,333
fashion,307
food,525
gaming,17
health and lifestyle,519
mom and children,152
sports,122
tech,368


In [ ]:
train_data_path = "training-dataset.jsonl.gz"

username2posts_train = dict()
username2profile_train = dict()

username2posts_test = dict()
username2profile_test = dict()




with gzip.open(train_data_path, "rt") as fh:
  for line in fh:
    sample = json.loads(line)

    profile = sample["profile"]
    username = profile["username"]
    if username in username2_category:
      # train data info
      username2posts_train[username] = sample["posts"]
      username2profile_train[username] = profile


    else:
      # it is test data info
      username2posts_test[username] = sample["posts"]
      username2profile_test[username] = profile


In [ ]:
# Profile Dataframe
train_profile_df = pd.DataFrame(username2profile_train).T.reset_index(drop=True)
test_profile_df = pd.DataFrame(username2profile_test).T.reset_index(drop=True)

train_profile_df.head(2)

,username,id,full_name,biography,category_name,post_count,follower_count,following_count,is_business_account,is_private,...,business_category_name,overall_category_name,category_enum,is_verified_by_mv4b,is_regulated_c18,profile_pic_url,should_show_category,should_show_public_contacts,show_account_transparency_details,profile_picture_base64
0,deparmedya,3170700063,Depar Medya,#mediaplanning #mediabuying #sosyalmedya,Local business,None,1167,192,True,False,...,None,None,LOCAL,False,False,https://instagram.fsaw2-3.fna.fbcdn.net/v/t51....,True,True,True,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
1,kafesfirin,266439571,KAFES FIRIN,📍Söğütözü📍FTZ AVM\n🛒Ankara macro▲center v...,Brand,None,11997,17,True,False,...,None,None,BRAND,False,False,https://instagram.fada1-13.fna.fbcdn.net/v/t51...,True,True,True,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...


In [ ]:
test_profile_df.head(2)

,username,id,full_name,biography,category_name,post_count,follower_count,following_count,is_business_account,is_private,...,business_category_name,overall_category_name,category_enum,is_verified_by_mv4b,is_regulated_c18,profile_pic_url,should_show_category,should_show_public_contacts,show_account_transparency_details,profile_picture_base64
0,beyazyakaliyiz,8634457436,Selam Beyaz Yakalı,Beyaz yakalıların dünyasına hoşgeldiniz 😀😀😀,Personal blog,None,1265,665,True,False,...,None,None,PERSONAL_BLOG,False,False,https://instagram.fist6-1.fna.fbcdn.net/v/t51....,True,True,True,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
1,totalenergies_istasyonlari,7066643793,TotalEnergies İstasyonları,TotalEnergies İstasyonları resmi Instagram hes...,Energy Company,None,28025,4,True,False,...,None,None,ENERGY_COMPANY,False,False,https://instagram.fsaw2-1.fna.fbcdn.net/v/t51....,True,True,True,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...


In [ ]:

def preprocess_text(text: str):
    # lower casing Turkish Text, Don't use str.lower :)
    text = text.casefold()

    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

    # Remove special characters and punctuation
    # HERE THE EMOJIS stuff are being removed, you may want to keep them :D
    text = re.sub(r'[^a-zçğıöşü0-9\s#@]', '', text)

    # Remove numbers
    text = re.sub(r'\d+', '', text)

    # Remove extra whitespaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text


corpus = []

# to keep the label order
train_usernames = []

for username, posts in username2posts_train.items():
  train_usernames.append(username)

  # aggregating the posts per user
  cleaned_captions = []
  for post in posts:
    post_caption = post.get("caption", "")
    if post_caption is None:
      continue

    post_caption = preprocess_text(post_caption)

    if post_caption != "":
      cleaned_captions.append(post_caption)


  # joining the posts of each user with a \n
  user_post_captions = "\n".join(cleaned_captions)
  corpus.append(user_post_captions)


vectorizer = TfidfVectorizer(stop_words=turkish_stopwords, max_features=5000)

# fit the vectorizer
vectorizer.fit(corpus)


# transform the data into vectors
x_post_train = vectorizer.transform(corpus)
y_train = [username2_category.get(uname, "NA") for uname in train_usernames]


test_usernames = []
test_corpus = []
for username, posts in username2posts_test.items():
  test_usernames.append(username)
  # aggregating the posts per user
  cleaned_captions = []
  for post in posts:
    post_caption = post.get("caption", "")
    if post_caption is None:
      continue

    post_caption = preprocess_text(post_caption)

    if post_caption != "":
      cleaned_captions.append(post_caption)

  user_post_captions = "\n".join(cleaned_captions)
  test_corpus.append(user_post_captions)


# Just transforming! No Fitting!!!!!
x_post_test = vectorizer.transform(test_corpus)

I addded metadata columns, such as follower_count, following_count, is_business_account, is_private, and is_verified, provide additional information about users that can significantly improve model performance.Adding these metadata columns makes the model more robust by incorporating user-level features alongside post-level textual data (TF-IDF vectors).



In [ ]:


# Define metadata columns to add
columns_to_add = ['follower_count', 'following_count', 'is_business_account', 'is_private', 'is_verified']

# Create DataFrames for metadata features (example for train and test)
train_profile_df = pd.DataFrame(username2profile_train).T.reset_index(drop=True)  # Adjust based on your train profile data
test_profile_df = pd.DataFrame(username2profile_test).T.reset_index(drop=True)    # Adjust based on your test profile data

# Combine TF-IDF vectors with metadata for train
df_tfidf_train = pd.concat(
    [
        pd.DataFrame(x_post_train.toarray(), columns=vectorizer.get_feature_names_out()),
        train_profile_df[columns_to_add].reset_index(drop=True),
    ],
    axis=1
)

# Combine TF-IDF vectors with metadata for test
df_tfidf_test = pd.concat(
    [
        pd.DataFrame(x_post_test.toarray(), columns=vectorizer.get_feature_names_out()),
        test_profile_df[columns_to_add].reset_index(drop=True),
    ],
    axis=1
)

# Handle missing values in the metadata for train
for col in columns_to_add:
    if df_tfidf_train[col].dtype.kind in 'biufc':  # Numeric columns
        df_tfidf_train[col].fillna(df_tfidf_train[col].mean(), inplace=True)
    else:  # Non-numeric columns
        df_tfidf_train[col].fillna(False, inplace=True)

# Handle missing values in the metadata for test
for col in columns_to_add:
    if df_tfidf_test[col].dtype.kind in 'biufc':  # Numeric columns
        df_tfidf_test[col].fillna(df_tfidf_test[col].mean(), inplace=True)
    else:  # Non-numeric columns
        df_tfidf_test[col].fillna(False, inplace=True)

# Example output check
print(df_tfidf_train.head())
print(df_tfidf_test.head())


   abdullah  abone  about  acele  acil  activities  acı   ad  ada  adam  ...  \
0  0.000000    0.0    0.0    0.0   0.0         0.0  0.0  0.0  0.0   0.0  ...   
1  0.000000    0.0    0.0    0.0   0.0         0.0  0.0  0.0  0.0   0.0  ...   
2  0.000000    0.0    0.0    0.0   0.0         0.0  0.0  0.0  0.0   0.0  ...   
3  0.046529    0.0    0.0    0.0   0.0         0.0  0.0  0.0  0.0   0.0  ...   
4  0.000000    0.0    0.0    0.0   0.0         0.0  0.0  0.0  0.0   0.0  ...   

   şükranla  şükür      şık  şıklık  şıklığı  follower_count  following_count  \
0       0.0    0.0  0.04874     0.0      0.0            1167              192   
1       0.0    0.0  0.00000     0.0      0.0           11997               17   
2       0.0    0.0  0.00000     0.0      0.0            2321              454   
3       0.0    0.0  0.00000     0.0      0.0           13647               29   
4       0.0    0.0  0.00000     0.0      0.0            7917               11   

   is_business_account  is_priva

<ipython-input-164-0710e8a5d7e1>:31: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_tfidf_train[col].fillna(False, inplace=True)
<ipython-input-164-0710e8a5d7e1>:38: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try u

In [ ]:
# Making sure everything is fine
assert y_train.count("NA") == 0

In [ ]:
feature_names = vectorizer.get_feature_names_out()
feature_names

array(['abdullah', 'abone', 'about', ..., 'şık', 'şıklık', 'şıklığı'],
      dtype=object)

In [ ]:
df_tfidf = pd.DataFrame(x_post_train.toarray(), columns=feature_names)
df_tfidf.head(2)

,abdullah,abone,about,acele,acil,activities,acı,ad,ada,adam,...,şubemiz,şubesi,şölen,şöleni,şöyle,şükranla,şükür,şık,şıklık,şıklığı
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.04874,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0


In [ ]:
df_tfidf.shape

(2807, 5000)

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(df_tfidf, y_train, test_size=0.2, stratify=y_train)

In [ ]:
x_train.shape

(2245, 5000)

In [ ]:
x_val.shape

(562, 5000)

#**LOGISTIC REGRESSION**

Logistic Regression is a practical and effective choice, especially when combined with features like TF-IDF vectors and metadata. It leverages both text and metadata features for accurate classification. Moreover, I tried Random Forest however validation was rather low.

In [ ]:
# Initialize the Logistic Regression model
clf = LogisticRegression(max_iter=1000, random_state=42)

# Train the model on the training data
clf.fit(x_train, y_train)

# Check training performance
train_accuracy = clf.score(x_train, y_train)
print(f"Training Accuracy: {train_accuracy:.2f}")


Training Accuracy: 0.84


In [ ]:
# Predict categories for the validation set
y_val_pred = clf.predict(x_val)

# Compute validation accuracy
val_accuracy = accuracy_score(y_val, y_val_pred)
print(f"Validation Accuracy: {val_accuracy:.2f}")

# Print detailed classification report
print("\nClassification Report:")
print(classification_report(y_val, y_val_pred))


Validation Accuracy: 0.65

Classification Report:
                      precision    recall  f1-score   support

                 art       0.64      0.18      0.28        39
       entertainment       0.47      0.50      0.49        66
             fashion       0.59      0.62      0.61        61
                food       0.83      0.91      0.87       104
              gaming       0.00      0.00      0.00         3
health and lifestyle       0.56      0.78      0.65       103
    mom and children       1.00      0.20      0.33        30
              sports       1.00      0.46      0.63        24
                tech       0.65      0.74      0.69        72
              travel       0.72      0.72      0.72        60

            accuracy                           0.65       562
           macro avg       0.65      0.51      0.53       562
        weighted avg       0.67      0.65      0.63       562



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


#**TEST**

In [ ]:
# Load test file (e.g., test-classification-round1.dat)
test_file_path = "test-classification-round1.dat"
with open(test_file_path, "r") as file:
    test_usernames = [line.strip() for line in file.readlines()]

# Prepare the test corpus
test_corpus = []
for username in test_usernames:
    posts = username2posts_test.get(username, [])
    cleaned_captions = []
    for post in posts:
        post_caption = post.get("caption", "")
        if post_caption is None:
            continue
        post_caption = preprocess_text(post_caption)
        if post_caption != "":
            cleaned_captions.append(post_caption)
    user_post_captions = "\n".join(cleaned_captions)
    test_corpus.append(user_post_captions)

# Transform the test corpus using the TF-IDF vectorizer
x_post_test = vectorizer.transform(test_corpus)

# Predict categories using the trained model
y_test_pred = clf.predict(x_post_test)

# Map usernames to their predicted categories
test_predictions = {uname: pred for uname, pred in zip(test_usernames, y_test_pred)}

# Save predictions to a JSON file
output_file = "prediction-classification-round1.json"
with open(output_file, "w") as f:
    json.dump(test_predictions, f)

print(f"Test predictions saved to {output_file}")


Test predictions saved to prediction-classification-round1.json


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


#Task-2

This function reads a .jsonl file (not gzipped).

In [ ]:
import json

def read_jsonl(file_path):
    """
    Reads a JSONL file and returns a list of dictionaries.
    """
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                data.append(json.loads(line.strip()))
            except json.JSONDecodeError:
                print(f"Skipping invalid JSON line: {line.strip()}")
    return data


In [ ]:
import gzip
import json

def read_jsonl_gz(file_path):
    """
    Reads a gzipped JSONL file and returns a list of dictionaries.
    """
    data = []
    try:
        with gzip.open(file_path, 'rt', encoding='utf-8') as f:
            for line in f:
                try:
                    data.append(json.loads(line.strip()))
                except json.JSONDecodeError as e:
                    print(f"Skipping invalid JSON line: {line.strip()} | Error: {e}")
    except FileNotFoundError:
        print(f"Error: The file '{file_path}' was not found. Please check the path.")
    return data


This function extracts features for each post from the data.

In [ ]:
def prepare_features(data_entries):
    """
    Prepare features for test data.
    Extracts features such as hashtag count, caption length, etc.
    """
    features = []
    for entry in data_entries:
        user_id = entry.get("profile", {}).get("username", "unknown")
        posts = entry.get("posts", [])

        for post in posts:
            # Extract features
            post_id = post.get("id", "")
            caption = post.get("caption", "")
            hashtag_count = caption.count("#")
            caption_length = len(caption.split())
            comments_count = post.get("comments_count", 0)
            like_count = post.get("like_count", 0)

            # Add features
            features.append({
                "user_id": user_id,
                "post_id": post_id,
                "hashtag_count": hashtag_count,
                "caption_length": caption_length,
                "comments_count": comments_count,
                "like_count": like_count
            })

    # Convert to DataFrame
    feature_df = pd.DataFrame(features)

    # Log-transform like_count for consistency
    if "like_count" in feature_df.columns:
        feature_df["log_like_count"] = np.log1p(feature_df["like_count"])

    return feature_df


Prepare and clean the training data for model training.

In [ ]:
import numpy as np

# Load training data
file_path = "training-dataset.jsonl.gz"  # Replace with the correct path
data_entries = read_jsonl_gz(file_path)

# Prepare features
post_feature_df = prepare_post_features(data_entries)

# Log-transform the target variable
post_feature_df["log_like_count"] = np.log1p(post_feature_df["like_count"])

# Handle missing values
post_feature_df.fillna(0, inplace=True)

# Print to verify
print(post_feature_df.info())
print(post_feature_df.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187302 entries, 0 to 187301
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   user_id         187302 non-null  object 
 1   post_id         187302 non-null  object 
 2   hashtag_count   187302 non-null  int64  
 3   caption_length  187302 non-null  int64  
 4   comments_count  187302 non-null  int64  
 5   like_count      187302 non-null  float64
 6   log_like_count  187302 non-null  float64
dtypes: float64(2), int64(3), object(2)
memory usage: 10.0+ MB
None
      user_id            post_id  hashtag_count  caption_length  \
0  deparmedya  17990918969458720              0               4   
1  deparmedya  18219250732221045              2               5   
2  deparmedya  18311380465102328              1               2   
3  deparmedya  18089518138361507              1               3   
4  deparmedya  18012743929758497              2               4   

   co

Train a regression model to predict the log-transformed like counts.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Separate features and target
X = post_feature_df.drop(columns=["like_count", "log_like_count", "user_id", "post_id"])
y = post_feature_df["log_like_count"]

# Split data
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the model
model = LinearRegression()
model.fit(X_train, y_train)

# Validate the model
y_val_pred = model.predict(X_val)
rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
print(f"Validation RMSE: {rmse:.2f}")

# Check training performance
y_train_pred = model.predict(X_train)
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
print(f"Training RMSE: {train_rmse:.2f}")


Validation RMSE: 2.42
Training RMSE: 2.42


In [ ]:
def prepare_post_features(data_entries):
    """
    Prepare features for test data.
    """
    features = []
    for entry in data_entries:
        user_id = entry.get("profile", {}).get("username", "unknown")
        posts = entry.get("posts", [])

        for post in posts:
            post_id = post.get("id", "")
            caption = post.get("caption", "") or ""
            hashtag_count = caption.count("#")
            caption_length = len(caption.split())
            comments_count = post.get("comments_count", 0)
            like_count = post.get("like_count", 0)

            features.append({
                "user_id": user_id,
                "post_id": post_id,
                "hashtag_count": hashtag_count,
                "caption_length": caption_length,
                "comments_count": comments_count,
                "like_count": like_count,
            })

    return pd.DataFrame(features)


In here I tried to use Random Forest but still had problem

In [338]:
import json
import gzip
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Function to read JSONL file
def read_jsonl(file_path):
    """
    Reads a JSONL file and returns a list of dictionaries.
    """
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                data.append(json.loads(line.strip()))
            except json.JSONDecodeError:
                print(f"Skipping invalid JSON line: {line.strip()}")
    return data

# Function to read gzipped JSONL file
def read_jsonl_gz(file_path):
    """
    Reads a gzipped JSONL file and returns a list of dictionaries.
    """
    data = []
    try:
        with gzip.open(file_path, 'rt', encoding='utf-8') as f:
            for line in f:
                try:
                    data.append(json.loads(line.strip()))
                except json.JSONDecodeError as e:
                    print(f"Skipping invalid JSON line: {line.strip()} | Error: {e}")
    except FileNotFoundError:
        print(f"Error: The file '{file_path}' was not found. Please check the path.")
    return data

# Function to prepare features from data entries
def prepare_features(data_entries):
    """
    Prepare features for test data.
    Extracts features such as hashtag count, caption length, etc.
    """
    features = []
    for entry in data_entries:
        user_id = entry.get("profile", {}).get("username", "unknown")
        posts = entry.get("posts", [])

        for post in posts:
            post_id = post.get("id", "")
            caption = post.get("caption", "")
            comments_count = post.get("comments_count", 0)
            like_count = post.get("like_count", 0)

            features.append({
                "user_id": user_id,
                "post_id": post_id,
                "comments_count": comments_count,
                "like_count": like_count
            })

    # Convert to DataFrame
    feature_df = pd.DataFrame(features)

    # Log-transform like_count for consistency
    if "like_count" in feature_df.columns:
        feature_df["log_like_count"] = np.log1p(feature_df["like_count"])

    return feature_df

# Load and prepare training data
file_path = "training-dataset.jsonl.gz"  # Replace with the correct path
data_entries = read_jsonl_gz(file_path)
post_feature_df = prepare_features(data_entries)

# Handle missing values: Replace NaN values in numeric columns with column averages
numeric_cols = post_feature_df.select_dtypes(include=[np.number]).columns
post_feature_df[numeric_cols] = post_feature_df[numeric_cols].fillna(post_feature_df[numeric_cols].mean())

# Separate features and target
X = post_feature_df.drop(columns=["like_count", "log_like_count", "user_id", "post_id"])
y = post_feature_df["log_like_count"]

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train Random Forest Regressor
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Validate the model
y_val_pred = model.predict(X_val)
rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
print(f"Validation RMSE: {rmse:.2f}")

# Function to predict on test data and save results
def predict_test_data(test_file_path, model):
    """
    Predicts like counts for test data and saves the results to a JSON file.
    """
    # Load the test dataset
    test_data_entries = read_jsonl(test_file_path)

    # Prepare features for the test data
    test_feature_df = prepare_features(test_data_entries)

    # Handle missing values: Replace NaN values in numeric columns with column averages
    numeric_cols = test_feature_df.select_dtypes(include=[np.number]).columns
    test_feature_df[numeric_cols] = test_feature_df[numeric_cols].fillna(test_feature_df[numeric_cols].mean())

    # Ensure required columns are present
    required_columns = X.columns
    for col in required_columns:
        if col not in test_feature_df.columns:
            test_feature_df[col] = 0  # Fill missing columns with default values

    # Extract features for prediction
    X_test = test_feature_df[required_columns]

    # Check if X_test is valid
    if X_test.empty or X_test.isnull().any().any():
        print("Error: Test feature dataframe is empty or contains NaN values.")
        return

    # Make predictions
    test_predictions_log = model.predict(X_test)
    test_predictions = np.expm1(test_predictions_log)  # Reverse log transformation

    # Extract post IDs
    post_ids = test_feature_df["post_id"].values if "post_id" in test_feature_df.columns else range(len(test_predictions))

    # Create a dictionary of predictions
    prediction_dict = {post_id: float(pred) for post_id, pred in zip(post_ids, test_predictions)}

    # Save predictions to a JSON file
    output_file = "prediction-regression-round3.json"  # Adjust filename as needed
    with open(output_file, "w") as f:
        json.dump(prediction_dict, f, indent=2)

    print(f"Predictions saved to: {output_file}")

# Test file path
test_file_path = "test-regression-round3.jsonl"  # Replace with your test data path

# Call the function to predict and save results
predict_test_data(test_file_path, model)


Validation RMSE: 1.28
Error: Test feature dataframe is empty or contains NaN values.


In [ ]:
print(test_feature_df.info())
print(test_feature_df.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Empty DataFrame
None
Empty DataFrame
Columns: []
Index: []
